# Homework 2

## Physics 7810
## Due February 10, 2020

## H2.1 - Molecular dynamics of the Lennard-Jones system

Write an MD program to simulate the three-dimensional (3D) Lennard-Jones (LJ) system in the $NVE$ ensemble in Python, with separate Python functions for initialization, the MD loop, integration of the equations of motion, the force calculation, etc. Implement periodic boundary conditions, assuming that the $N$ particles in the simulation are initially placed in a cubic box of sidelength $L$ (so the volume $V = L^3$).

Refer to Chapter 1 in Allen and Tildesley for the definition of the LJ potential (Section 1.3.2) and for a discussion of how to implement periodic boundary conditions (Section 1.6.4). To calculate the LJ interactions, you'll need to use the *nearest image convention*, which can be implemented in a similar way to periodic boundary conditions, and is discussed in Sections 1.6.3 and 1.6.4 of Allen and Tildesley.

This will be very similar in structure to the MD program you wrote for the second tutorial, but in 3D instead of 2D and with periodic boundary conditions, and with a different interaction potential (and no central force potential). 

To initialize the system, particles can be placed on a regular lattice, or placed at random points within cubic box, taking care to avoid close contacts between pairs of particles. Assign random velocities to the particles initially (sampled from either a uniform or Gaussian distribution). *Hint*: check out the `random` Python library for generating random deviates.

Using your simulation code, try to reproduce something resembling Figure 4.3 in Frenkel and Smit, which is a plot of the kinetic, potential, and total energies of the system as a function of the number of timesteps in a simulation of $N = 108$ LJ particles. You may need to vary the integration timestep to obtain stable dynamics. Frenkel and Smit say that this simulation was run at a *dimensionless temperature* of $T^\ast = 0.728$ and a *dimensionless density* of $\rho^\ast = 0.8442$. Try to figure out how these dimensionless thermodynamic variables are defined in terms of the parameters of the LJ potential.

We haven't yet discussed how to measure temperature, but you can obtain the average temperature using the equipartition theorem, which gives

$$
k_B T = \frac{\langle 2 K \rangle}{N_f}.
$$

Here, $K$ is the kinetic energy, $N_f = 3N - 3$ is the number of quadratic degrees of freedom that contribute to the kinetic energy, and the angle brackets denote a time average (see Chapter 2 in Allen and Tildesley or Chapter 4 in Frenkel and Smit).

An *instantaneous* temperature $T(t)$ can also be defined through

$$
k_B T(t) = \frac{\langle 2 K(t) \rangle}{N_f},
$$

where $K(t)$ is the instantaneous kinetic energy. Use these expressions to measure the instantaneous and time-averaged temperatures for your simulation of the LJ system, and make a plot of $T(t)$ as a function of $t$.